In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.colors import BoundaryNorm

In [2]:
# Check if the CONUS run ERW model and default model have the same vegetation status
# Yes
case_erw = '20240923_conus_ICB1850CNRDCTCBC_ad_spinup_erw'
case_default = '20240923_conus_ICB1850CNRDCTCBC_ad_spinup'
for case, name in zip([case_erw, case_default], ['ERW', 'No ERW']):
    hr2 = xr.open_dataset(
        os.path.join(os.environ['E3SM_ROOT'], 'output', case, 'run', 
                    f'{case}.elm.h1.0161-01-01-00000.nc')
    )
    temp = np.where((np.abs(hr2['pfts1d_lat'] - 37.25) < 1e-3) & 
                    (np.abs(hr2['pfts1d_lon'] - 274.75) < 1e-3))[0]
    print(name, hr2['GPP'].values[0, temp])
    hr2.close()

ERW [          nan 4.4570395e-05           nan           nan           nan
           nan           nan 4.6188004e-05           nan           nan
           nan           nan           nan 5.4091761e-05 7.7776400e-05
 5.5519220e-05           nan           nan           nan           nan
           nan           nan           nan           nan           nan
           nan           nan           nan           nan           nan
           nan           nan 0.0000000e+00]
No ERW [          nan 4.4570395e-05           nan           nan           nan
           nan           nan 4.6188004e-05           nan           nan
           nan           nan           nan 5.4091761e-05 7.7776400e-05
 5.5519220e-05           nan           nan           nan           nan
           nan           nan           nan           nan           nan
           nan           nan           nan           nan           nan
           nan           nan 0.0000000e+00]


In [3]:
case = '20240923_conus_ICB1850CNPRDCTCBC_erw'

hr0 = xr.open_dataset(
    os.path.join(os.environ['E3SM_ROOT'], 'output', case, 'run', 
                f'{case}.elm.r.0061-01-01-00000.nc')
)
cols1d_lon = hr0['cols1d_lon'].copy()
cols1d_lat = hr0['cols1d_lat'].copy()
topo1d_lon = hr0['topo1d_lon'].copy()
topo1d_lat = hr0['topo1d_lat'].copy()
pft1d_lon = hr0['pfts1d_lon'].copy()
pft1d_lat = hr0['pfts1d_lat'].copy()
mask = hr0['cols1d_ityp'].copy() == 1
mask_pft = hr0['pfts1d_active'].copy() > 0
hr0.close()

/ccsopen/home/ywo/.conda/envs/myCondaEnv/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'timemgr_rst_nstep_rad_prev' has multiple fill values {-9999, -999999999}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/ccsopen/home/ywo/.conda/envs/myCondaEnv/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'timemgr_rst_type' has multiple fill values {-9999, -999999999}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/ccsopen/home/ywo/.conda/envs/myCondaEnv/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'timemgr_rst_step_sec' has multiple fill values {-9999, -999999999}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/ccsopen/home/ywo/.conda/envs/myCondaEnv/lib/python3.11/site-packages/xarray/conventions.py:431: SerializationWarning: variable 'timemgr_rst_start_ymd' has multiple fill values {-9999, -999999999}, decoding all valu

In [6]:
#case = '20240923_conus_ICB1850CNRDCTCBC_ad_spinup_erw'
#prefix = 'ad_spinup'
#for year in range(21, 201, 20):
case = '20240923_conus_ICB1850CNPRDCTCBC_erw'
prefix = 'normal_spinup'
for year in range(21, 402, 20):
    hr = xr.open_dataset(
        os.path.join(os.environ['E3SM_ROOT'], 'output', case, 'run', 
                    f'{case}.elm.r.{year:04d}-01-01-00000.nc')
    )

    # Only do the below if checking difference
    """for var in hr.data_vars:
        if np.issubdtype(hr[var].dtype, np.number): # 'column' in hr[var].dims:
            temp = hr[var] # .where(mask)
            if np.nanmax(temp) > 0:
                print(var, np.nanmax(temp))"""


    layer = 0
    fig, axes = plt.subplots(6, 3, figsize = (20, 20), sharex = True, sharey = True)
    # pH
    ax = axes[0, 0]
    data = hr[f'ceca_col'][:, layer].where(mask)
    norm = BoundaryNorm(
        np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
        ncolors = 256, clip = False, extend = 'both'
    )
    cf = ax.scatter(cols1d_lon.where(mask), cols1d_lat.where(mask), 
                    c = data, s = 1, cmap = 'Reds', norm = norm)
    plt.colorbar(cf, ax = ax)
    ax.set_title('ceca_col')

    ax = axes[0, 1]
    data = hr[f'soil_pH'][:, layer].where(mask)
    norm = BoundaryNorm(
        np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
        ncolors = 256, clip = False, extend = 'both'
    )
    cf = ax.scatter(cols1d_lon.where(mask), cols1d_lat.where(mask), 
                    c = data, s = 1, cmap = 'Reds', norm = norm)
    plt.colorbar(cf, ax = ax)
    ax.set_title('soil pH')

    ax = axes[0, 2]
    data = hr[f'cec_proton_vr'][:, layer].where(mask)
    norm = BoundaryNorm(
        np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
        ncolors = 256, clip = False, extend = 'both'
    )
    cf = ax.scatter(cols1d_lon.where(mask), cols1d_lat.where(mask), 
                    c = data, s = 1, cmap = 'Reds', norm = norm)
    plt.colorbar(cf, ax = ax)
    ax.set_title('cec_proton_vr')

    # cations
    for icat in range(1,6):
        ax = axes[icat, 0]
        data = hr[f'cece_col_{icat}'][:, layer].where(mask)
        norm = BoundaryNorm(
            np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
            ncolors = 256, clip = False, extend = 'both'
        )
        cf = ax.scatter(cols1d_lon.where(mask), cols1d_lat.where(mask), 
                        c = data, s = 1, cmap = 'Reds', norm = norm)
        plt.colorbar(cf, ax = ax)
        ax.set_title(f'cece_col_{icat}')

        ax = axes[icat, 1]
        data = hr[f'cation_vr_{icat}'][:, layer].where(mask)
        norm = BoundaryNorm(
            np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
            ncolors = 256, clip = False, extend = 'both'
        )
        cf = ax.scatter(cols1d_lon.where(mask), cols1d_lat.where(mask), 
                        c = data, s = 1, cmap = 'Reds', norm = norm)
        plt.colorbar(cf, ax = ax)
        ax.set_title(f'cation_vr_{icat}')

        ax = axes[icat, 2]
        data = hr[f'cec_cation_vr_{icat}'][:, layer].where(mask)
        norm = BoundaryNorm(
            np.nanpercentile(data.values.reshape(-1), np.arange(5, 96, 5)),
            ncolors = 256, clip = False, extend = 'both'
        )
        cf = ax.scatter(cols1d_lon.where(mask), cols1d_lat.where(mask), 
                        c = data, s = 1, cmap = 'Reds', norm = norm)
        plt.colorbar(cf, ax = ax)
        ax.set_title(f'cec_cation_vr_{icat}')

    plt.savefig(os.path.join(os.environ['PROJDIR'], 'ERW_LDRD', 'results', 'plot_spinupDiags', 
                             f'{prefix}_year_{year:04d}.png'), dpi = 600., 
                             bbox_inches = 'tight')
    plt.close(fig)

    hr.close()